In [21]:
using Mongoc

client = Mongoc.Client("mongodb://localhost:27017/")

Mongoc.ping(client)

database =  client["harbintrips"]
collection = database["trips2_07"]







Collection(Database(Client(URI("mongodb://localhost:27017/")), "harbintrips"), "trips2_07")

In [8]:
using JLD2


In [22]:

jldfilepath = "/path/to/zhongtai/data/trips2/trips_150107.jld2" 

dataset = load(jldfilepath)
println
println(length(dataset["trips"]))



251601


In [23]:
trips = dataset["trips"]
trip = trips[1]
println(trip.lon)

[126.68132, 126.67669, 126.67118, 126.666824, 126.66343, 126.65931, 126.656906, 126.65229, 126.652885, 126.6568, 126.65741]


In [19]:
mutable struct Trip{T<:AbstractFloat}
    lon::Vector{T}
    lat::Vector{T}
    tms::Vector{T}
    devid::Int64
    roads::Vector{Any}
    time::Vector{Any}
    frac::Vector{Any}
    route::Vector{Any}
    route_heading::Vector{Any}
    route_geom::Matrix{Any}
end



In [24]:
using Mongoc
using BSON

# documents = [BSON.BSON(Dict(
#     "longitudes" => trip.lon,
#     "latitudes" => trip.lat,
#     "timestamp" => trip.tms,
#     "devid" => trip.devid,
#     "roads" => trip.roads,
#     "time" => trip.time,
#     "frac" => trip.frac,
#     "route" => trip.route,
#     "route_heading" => trip.route_heading,
#     "route_geom" => trip.route_geom
# )) for trip in trips]

for trip in trips
    converted_route_geom = [collect(row) for row in eachrow(trip.route_geom)]
    loaded_trip = Dict(
        "longitudes" => Vector{Float64}(trip.lon),
        "latitudes" =>  Vector{Float64}(trip.lat),
        "timestamp" =>  Vector{Int64}(trip.tms),
        "devid" => Int64(trip.devid),
        "roads" => Vector{Any}(trip.roads),
        "time" => Vector{Any}(trip.time),
        "frac" => Vector{Any}(trip.frac),
        "route" => Vector{Any}(trip.route),
        "route_heading" => Vector{Any}(trip.route_heading),
        "route_geom" => converted_route_geom
    )
    bson_trip = Mongoc.BSON(loaded_trip)
    Mongoc.insert_one(collection, bson_trip)
end

println("Inserted $(length(documents)) documents")

UndefVarError: UndefVarError: `documents` not defined in `Main`
Suggestion: check for spelling errors or missing imports.